In [1]:
import syft as sy
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import torch.autograd as autograd

syft = sy 

hook = sy.TorchHook(torch)
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
charlie = sy.VirtualWorker(hook, id="charlie")
james = sy.VirtualWorker(hook, id="james")
crypto_provider = james

In [2]:
from syft.messaging import Plan

Plan.__init__(self,
            owner=sy.local_worker,
            id=sy.ID_PROVIDER.pop(),
            blueprint=self,
            name=self.__class__.__name__,
            is_method=True,
        )

In [3]:
class Net(nn.Module, Plan):
    def __init__(self):
        nn.Module.__init__(self)
        Plan.__init__(self,
            owner=sy.local_worker,
            id=sy.ID_PROVIDER.pop(),
            name=self.__class__.__name__,
            is_method=False,
        )
        self.fc1 = nn.Linear(2, 3)
        self.bias = torch.tensor([1000.])

        self.state += ['fc1', 'bias']

    def forward(self, x):
        x = self.fc1(x)
        return x + self.bias


In [4]:
net = Net()
t = torch.tensor([2., 4.])
r = Plan.__call__(net, t)
r

tensor([999.3454, 995.7429, 998.9573], grad_fn=<AddBackward0>)

In [5]:
net = Net()

In [6]:
r = net.build(torch.tensor([1., 2.]))

$ _build
weight id 86626019585


In [7]:
net.state_ids

[86626019585, 83195948792, 95995047861]

In [8]:
net.bias

tensor([1000.])

In [9]:
net.fc1.weight

Parameter containing:
tensor([[-0.6905, -0.4039],
        [ 0.5944,  0.1373],
        [ 0.1418, -0.5717]], requires_grad=True)

Parameter containing:
(Wrapper)>[PointerTensor | me:36668126710 -> me:35148386835]

In [10]:
net.readable_plan

((26,
  (1,
   ((6,
     ((5, (b'torch.nn.functional.linear',)),
      None,
      (6,
       ((20, (44534359208, 41629293271, 'me', None, (10, (2,)), True)),
        (20, (35316758448, 86626019585, 'me', None, (10, (3, 2)), True)),
        (20, (29691047349, 83195948792, 'me', None, (10, (3,)), True)))),
      (0, ()))),
    (24072195705,)))),
 (26,
  (1,
   ((6,
     ((5, (b'__add__',)),
      (20, (68797362662, 24072195705, 'me', None, None, True)),
      (6, ((20, (18371348318, 95995047861, 'me', None, (10, (1,)), True)),)),
      (0, ()))),
    (65769360457,)))))

In [11]:
net.result_ids

(65769360457,)

In [12]:
t = torch.tensor([2., 4.])
Plan.__call__(net, t)

$ __call__


tensor([ 997.4067, 1001.0394,  998.2351], grad_fn=<AddBackward0>)

### Send 1 time
Manual ADD to serde

In [13]:
sy.serde.simplifiers[Net] = (35, Plan.simplify)
sy.serde.detailers[35] = Plan.detail

In [14]:
Plan.send(net, bob)

$ modle send
$ replace_ids


Net(
  (fc1): Linear(in_features=2, out_features=3, bias=True)
)

In [15]:
bob._objects[net.ptr_plans['bob'].id].arg_ids

[41629293271]

In [16]:
bob._objects[net.ptr_plans['bob'].id].state_ids

[]

In [17]:
bob._objects[net.ptr_plans['bob'].id].readable_plan

((26,
  (1,
   ((6,
     ((5, (b'torch.nn.functional.linear',)),
      None,
      (6,
       ((20, (44534359208, 41629293271, b'bob', None, (10, (2,)), True)),
        (20, (35316758448, 73311733551, b'bob', None, (10, (3, 2)), True)),
        (20, (29691047349, 54311095995, b'bob', None, (10, (3,)), True)))),
      (0, ()))),
    (24072195705,)))),
 (26,
  (1,
   ((6,
     ((5, (b'__add__',)),
      (20, (68797362662, 24072195705, b'bob', None, None, True)),
      (6, ((20, (18371348318, 68623651823, b'bob', None, (10, (1,)), True)),)),
      (0, ()))),
    (65769360457,)))))

In [18]:
net.fc1.weight

Parameter containing:
(Wrapper)>[PointerTensor | me:46152201382 -> bob:73311733551]

In [19]:
net.bias

(Wrapper)>[PointerTensor | me:31559014361 -> bob:68623651823]

In [20]:
p = torch.tensor([2., 4.]).send(bob)
p

(Wrapper)>[PointerTensor | me:18310705707 -> bob:55575354719]

In [21]:
net.is_built

True

In [22]:
net.id

13758400870

In [23]:
r = Plan.__call__(net, p)

$ __call__
$ _execute_readable_plan
$ execute_plan
$ request_execute_plan
$ execute_plan
*** update args [41629293271] [55575354719]
$ replace_ids
$ replace_ids
$ _execute_plan


In [24]:
r.get()

tensor([ 997.4067, 1001.0394,  998.2351], requires_grad=True)


## Call back

In [25]:
Plan.__str__(net)

'<Net Net id:13758400870 owner:me location:bob built>'

In [26]:
Plan.get(net)

Net(
  (fc1): Linear(in_features=2, out_features=3, bias=True)
)

In [27]:
net.bias

tensor([1000.])

In [28]:
net.fc1.weight

Parameter containing:
tensor([[-0.6905, -0.4039],
        [ 0.5944,  0.1373],
        [ 0.1418, -0.5717]], requires_grad=True)

In [29]:
t = torch.tensor([2., 4.])
net(t)

tensor([ 997.4067, 1001.0394,  998.2351], grad_fn=<AddBackward0>)

In [30]:
Plan.__call__(net, t)

$ __call__


tensor([ 997.4067, 1001.0394,  998.2351], grad_fn=<AddBackward0>)

## Re-send

In [31]:
Plan.send(net, alice)

$ modle send
$ replace_ids


Net(
  (fc1): Linear(in_features=2, out_features=3, bias=True)
)

In [32]:
p = torch.tensor([2., 4.]).send(alice)

In [33]:
r = Plan.__call__(net, p)

$ __call__
$ _execute_readable_plan
$ execute_plan
$ request_execute_plan
$ execute_plan
*** update args [41629293271] [32086217866]
$ replace_ids
$ replace_ids
$ _execute_plan


In [34]:
r.get()

tensor([ 997.4067, 1001.0394,  998.2351], requires_grad=True)